# 0. Import package and Define

validation data없이 training data 420개를 사용하여 진행한다.

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <time.h>   
#include <string.h>


#define numInputs 256
#define imageWidth 16
#define imageHeight 16
#define numHiddenNodes 128
#define numOutputs 7
#define numTrainingSets 420
#define train_size 420
#define test_size 20
#define batch 128

# 1. read CSV data 
# (Triaining data, Test data) 

In [ ]:
typedef struct {
    int data[16][16];
} Image;

void read_dataset(const char* filename, int image[imageWidth][imageHeight]) {    // 16*16 csv파일 한 개, 즉 이미지 1개를 읽어들여 image 이차원배열에 저장
    FILE* file = fopen(filename, "r");
    if (!file)      exit(1);

    // CSV 파일에서 이미지 읽어오기
    for (int i = 0; i < 16; i++) {
        for (int j = 0; j < 16; j++) {
            if (fscanf(file, "%d,", &image[i][j]) != 1)     exit(1);
        }
    }
    fclose(file);
}

void read_trainlabel(const char* filename, int Y[numTrainingSets]) {    // 레이블 csv파일 한 개, 즉 전체 레이블을 읽어들여 Y배열에 저장
    FILE* file = fopen(filename, "r");
    if (file == NULL)   return;

    char line[1000];
    int index = 0;

    if (fgets(line, sizeof(line), file) != NULL) {
        line[strcspn(line, "\n")] = 0;
        char* token = strtok(line, ",");
        while (token != NULL && index < numTrainingSets) {
            Y[index] = atoi(token);
            index++;
            token = strtok(NULL, ",");
        }
    }
    fclose(file);
}
void read_testlabel(const char* filename, int Y[test_size]) {
    FILE* file = fopen(filename, "r");
    if (file == NULL)   return;

    char line[1000];
    int index = 0;

    if (fgets(line, sizeof(line), file) != NULL) {
        // Remove newline character at the end of the line
        line[strcspn(line, "\n")] = 0;

        // Tokenize the line using commas
        char* token = strtok(line, ",");
        while (token != NULL && index < test_size) {
            // Convert string to integer and store in Y array
            Y[index] = atoi(token);
            index++;

            // Move to the next token
            token = strtok(NULL, ",");
        }
    }

    fclose(file);
}

# 3. define function
### (sigmoid, softmax, initialization, dotproduct, transpose, ...)
###### Activation 함수로 sigmoid, softmax를 정의하고
###### 초기 weight값을 설정하기 위한 init 함수를 정의한다.
###### 추가로 c언어에서 행렬 연산을 위한 flatten, transpose, dot product 함수를 정의한다.

In [ ]:
void Sigmoid(double* z, double* output, int size) {     // 배열 z를 받아 sigmoid한 결과를 output에 저장하는 Sigmoid
    for (int i = 0; i < size; i++) {
        output[i] = 1.0 / (1.0 + exp(-z[i]));
    }
}
void dSigmoid(double* z, double* output, int size) {    // 배열 output을 받아 sigmoid의 미분 값을 z에 저장하는 dSigmoid
    for (int i = 0; i < size; i++) {
        z[i] = output[i] * (1.0 - output[i]);
    }
}

void Softmax(double *z, double *output, int size) {     // 배열 z를 받아 softmax한 결과를 output에 저장하는 Softmax
    double max_val = z[0];
    for (int i = 1; i < size; i++) {
        if (z[i] > max_val) {
            max_val = z[i];
        }
    }

    double sum_exp = 0.0;
    for (int i = 0; i < size; i++) {
        output[i] = exp(z[i] - max_val);  // 오버플로우 방지를 위해 최대값으로 빼줌
        sum_exp += output[i];
    }

    for (int i = 0; i < size; i++) {
        output[i] /= sum_exp;
    }
}
void dSoftmax(double* z, double* out, double* _targets, int size) {     // 배열 output을 받아 softmax의 미분 값을 z에 저장하는 dSoftmax
    for (int i = 0; i < size; i++) {
        z[i] = 2 * (out[i] - _targets[i]) / size;
    }
}

void init1(double layer[numInputs][numHiddenNodes], int rows, int cols) {       // input layer와 hidden layer 사이의 가중치값을 초기화하는 함수
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            layer[i][j] = ((double)rand() / RAND_MAX) * 2.0 - 1.0;  // -1부터 1 사이의 난수로 초기화
        }
    }
}
void init2(double layer[numHiddenNodes][numOutputs], int rows, int cols) {      // hidden layer와 output layer 사이의 가중치값을 초기화하는 함수
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            layer[i][j] = ((double)rand() / RAND_MAX) * 2.0 - 1.0;  // -1부터 1 사이의 난수로 초기화
        }
    }
}


void flattenMatrix(double* dest, double*** src, int rows, int cols) {       // python에서 reshape를 통해 차원을 줄일 수 있는데, 해당 기능을 함수로 구현
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            dest[i * cols + j] = (*src)[i][j];
        }
    }
}
void shuffleArray(int* array, int size) {                                   // batch로 들어가는 값을 랜덤으로 정해주기 위해 사용하는 shuffleArray 함수
    srand((unsigned int)time(NULL));
    for (int i = size - 1; i > 0; i--) {
        int j = rand() % (i + 1);
        // Swap array[i] and array[j]
        int temp = array[i];
        array[i] = array[j];
        array[j] = temp;
    }
}

void dot(double* result, double* matrix1, double* matrix2, int rows1, int cols1, int cols2) {       // python에서 dot() 기능을 함수로 구현
                                                                                                    // matrix1과 matrix2를 dot product하여 result로
    for (int i = 0; i < rows1; i++) {
        for (int j = 0; j < cols2; j++) {
            result[i * cols2 + j] = 0;
            for (int k = 0; k < cols1; k++) {
                result[i * cols2 + j] += matrix1[i * cols1 + k] * matrix2[k * cols2 + j];
            }
        }
    }
}
void transposeMatrix(double* result, double* matrix, int rows, int cols) {      // python에서 .T 기능을 함수로 구현
                                                                                // matrix를 transpose하여 result로 저장
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            result[j * rows + i] = matrix[i * cols + j];
        }
    }
}


// 전역변수 선언 (stack overflow 방지)
int X_train[420][imageWidth][imageHeight];
int X_test[20][imageWidth][imageHeight];

double w1[numInputs][numHiddenNodes];
double w2[numHiddenNodes][numOutputs];

double d_w1[numInputs][numHiddenNodes];
double d_w2[numHiddenNodes][numOutputs];

## 4. forward and backward pass & Training

##### main에서 순전파, 역전파를 진행한다.

1. 학습데이터와 테스트데이터, 각각의 레이블을 불러온다.
2. 가중치 값을 초기화한다.
3. 학습한다.

In [ ]:
int main(void) {

    // 1. read data
    // 420개의 16*16 크기의 csv 학습 데이터셋을 읽는다.
    //  20개의 16*16 크기의 csv 테스트 데이터셋을 읽는다.
    for (int i = 0; i < 420; i++) {     // 420개의 트레이닝 이미지를 불러옴
        char filename[256];
        sprintf(filename, "train_dataset_csv/%d.csv", i + 1);
        read_dataset(filename, X_train[i]);
    }

    for (int i = 1; i <= 20; i++) {     // 20개의 테스트 이미지를 불러옴
        char filename[50];
        snprintf(filename, sizeof(filename), "test_dataset_csv/test%d.csv", i);
        read_dataset(filename, X_test[i - 1]);
    }

    // 420*1 크기의 csv 학습 레이블을 읽는다.
    //  20*1 크기의 csv 테스트 레이블을 읽는다.
    int Y_train[420] = { 0, };
    int Y_test[20] = { 0, };

    read_trainlabel("train_label.csv", Y_train);    // 트레이닝 데이터의 레이블을 불러온다 (420,1)  
    read_testlabel("test_label.csv", Y_test);       // 학습 데이터의 레이블을 불러온다 (20,1)


   // 2. Initializing weights
   // input layer와 hidden layer 사이의 가중치 w1, 
   // hidden layer와 output layer 사이의 가중치 w2를 랜덤값으로 생성한다.
    srand(42);  // 랜덤값 생성을 위해 선언

    init1(w1, numInputs, numHiddenNodes);
    init2(w2, numHiddenNodes, numOutputs);


    // 3. Training
    // 3-1. 128크기의 배치로 한 epoch에 128개의 사진을 사용하여 벡터화하여 한 번에 처리한다. (batch = 128)
    // 3-2. 순전파
    // 3-3. 역전파

    int epochs = 18000;          // num of epochs
    const double lr = 0.001;     // learning rate

    for (int e = 0; e < epochs; e++) {

        // 3-1. batch 사이즈에 맞게 랜덤으로 값을 뽑아 x,y에 저장하고 타겟값을 생성한다.
        double targets[batch][numOutputs] = {0, };
        double x[batch][numInputs] = {0, };
        double y[batch] = {0, };

    
        int rand[train_size] = { 0, };
        for (int i = 0; i < train_size; i++)  rand[i] = i;
        shuffleArray(rand, train_size);

        for (int i = 0; i < batch; i++) {
            int index = rand[i];
            for (int j = 0; j < 16; j++) {
                for (int k = 0; k < 16; k++) {
                    x[i][j * 16 + k] = X_train[index][j][k];
                }
            }
            y[i] = Y_train[index];
        }


        // 3-2. Forward pass
        double z1[batch][numHiddenNodes];
        double a1[batch][numHiddenNodes];
        double z2[batch][numOutputs];
        double output[batch][numOutputs];

        dot(&z1[0][0], &x[0][0], &w1[0][0], batch, numInputs, numHiddenNodes);
        Sigmoid(&z1[0][0], &a1[0][0], batch * numHiddenNodes);
        dot(&z2[0][0], &a1[0][0], &w2[0][0], batch, numHiddenNodes, numOutputs);
        Softmax(&z2[0][0], &output[0][0], batch * numOutputs);


        // 3-3. Backpropagation
        double error[batch][numOutputs] = {0, };

        for (int i = 0; i < batch; i++) {
            dSoftmax(&error[i][0], &output[i][0], &targets[i][0], numOutputs);
        }

        transposeMatrix(&a1[0][0], &a1[0][0], batch, numHiddenNodes);
        dot(&d_w2[0][0], &a1[0][0], &error[0][0], numHiddenNodes, batch, numOutputs);

        dot(&error[0][0], &w2[0][0], &error[0][0], batch, numOutputs, numHiddenNodes);
        for (int i = 0; i < batch; i++) {
            dSigmoid(&a1[i][0], &a1[i][0], numHiddenNodes);
        }
        for (int i = 0; i < batch; i++) {
            for (int j = 0; j < numHiddenNodes; j++) {
                error[i][j] *= a1[i][j];
            }
        }

        transposeMatrix(&x[0][0], &x[0][0], batch, numInputs);
        dot(&d_w1[0][0], &x[0][0], &error[0][0], numInputs, batch, numHiddenNodes);

    }   
    
    return 0;
}

## 6. Testing